<a href="https://colab.research.google.com/github/HossamSaoud/MachineLearning_Notebooks/blob/main/Company_brochure_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [7]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import userdata

In [11]:
# Initialize and constants

api_key = userdata.get("OPENAI_API_KEY")

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI(api_key=api_key)

API key looks good so far


In [12]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [13]:
ed = Website("https://www.sixt.com/")
ed.links

['/',
 '/truck-rental/',
 'https://www.sixt.com/campaign/losangeles-relief-us/?promo_name=LAwildfirerelief_15&promo_creative=heart&promo_position=sixt.com',
 'https://www.sixt.com/campaign/rent-winter-promotion-ice/?promo_name=winterspecial_15&promo_creative=winterspecial&promo_position=sixt.com',
 'https://www.sixt.com/campaign/5-luxuryspecial-us/?promo_name=Luxuryspecial_15&promo_creative=luxuryspecial&promo_position=sixt.com',
 'https://corporate-signup.sixt.com/?lang=en&country_code=us&product=undefined&utm_source=sixt.com&utm_medium=retail-website-referral&utm_campaign=US_US_btob_HOME-DEALS_RAC_ST_ENG&utm_content=Box-business&utm_term=b2b_ref',
 'https://www.sixt.com/ride/campaign?campaignId=5-en-us-summer-promo&title=under&utm_source=rent_hp_referral_ride&utm_medium=sixt_referral&utm_campaign=deals&promo_name=blackfriday_20&promo_creative=deals&promo_position=sixt.com',
 '/plus/?promo_name=rent_hp_referral_plus&promo_creative=more_sixt&promo_position=sixt.com',
 'https://corporat

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [14]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [15]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [16]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [17]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://www.sixt.com/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/truck-rental/
https://www.sixt.com/campaign/losangeles-relief-us/?promo_name=LAwildfirerelief_15&promo_creative=heart&promo_position=sixt.com
https://www.sixt.com/campaign/rent-winter-promotion-ice/?promo_name=winterspecial_15&promo_creative=winterspecial&promo_position=sixt.com
https://www.sixt.com/campaign/5-luxuryspecial-us/?promo_name=Luxuryspecial_15&promo_creative=luxuryspecial&promo_position=sixt.com
https://corporate-signup.sixt.com/?lang=en&country_code=us&product=undefined&utm_source=sixt.com&utm_medium=retail-website-referral&utm_campaign=US_US_btob_HOME-DEALS_RAC_ST_ENG&utm_content=Box-business&utm_term=b2b_ref
https://www.sixt.com/ride/campaign?campaignId=5-en-us-summer-promo&titl

In [18]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
print(get_all_details("https://www.sixt.com/"))

Found links: {'links': [{'type': 'about page', 'url': 'https://about.sixt.com/'}, {'type': 'newsroom page', 'url': 'https://about.sixt.com/en/newsroom/'}, {'type': 'investor relations page', 'url': 'https://about.sixt.com/en/investor-relations/'}, {'type': 'careers page', 'url': 'https://www.sixt.jobs/us'}]}
Landing page:
Webpage Title:
SIXT rent a car | Premium Car Rental & Top Deals Worldwide
Webpage Contents:
112 years of SIXT. 112 years of tradition.
SIXT
Cars
Trucks
Pick-up & return
Different return location
Pick-up date
Return date
Show cars
Airport, city or address
Select pickup
Don't rent a car.
Rent THE car.
Premium car rental at affordable rates. Worldwide.
Global reach
2,000+ SIXT stations in over 105 countries
Distinctive fleet
From high-end convertibles to premium SUVs
Exceptional service
Stress-free, trustworthy, no hidden costs
Up to
25%
discount
Los Angeles Wildfire Response
Discounted Long-term Rentals for People Impacted by the LA Wildfires
Find out more
Up to
15%
dis

In [21]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://www.sixt.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://about.sixt.com/'}, {'type': 'newsroom', 'url': 'https://about.sixt.com/en/newsroom/'}, {'type': 'investor relations', 'url': 'https://about.sixt.com/en/investor-relations/'}, {'type': 'careers page', 'url': 'https://www.sixt.jobs/us'}, {'type': 'corporate page', 'url': 'https://corporate.sixt.com/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nSIXT rent a car | Premium Car Rental & Top Deals Worldwide\nWebpage Contents:\n112 years of SIXT. 112 years of tradition.\nSIXT\nCars\nTrucks\nPick-up & return\nDifferent return location\nPick-up date\nReturn date\nShow cars\nAirport, city or address\nSelect pickup\nDon't rent a car.\nRent THE car.\nPremium car rental at affordable rates. Worldwide.\nGlobal reach\n2,000+ SIXT stations in over 105 countries\nDistinctive fleet\nFrom high-end convertibles to premium SUVs\nExceptional service\nStress-free, trustworthy, no hidden costs\nUp to\n25%\ndiscount\nLos Angeles Wildfire Response\nDiscounted Long-term Rentals for People Impacted by the LA Wildfires\nFind out more\nUp to\n15%\ndiscount\nThis one's ice & easy\nSave up to 15% worldwide on your winter rental from S

In [24]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [26]:
stream_brochure("sixt", "https://www.sixt.com/")

<generator object stream_brochure at 0x7a40087fca60>

In [28]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [29]:
import gradio as gr
view = gr.Interface(
    fn=stream_brochure,
    inputs = [gr.Textbox(label = "Company Name"), gr.Textbox(label="URL")],
    outputs = [gr.Markdown(label = "Response")],
    flagging_mode = "never"
)
view.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f125fc4c893371bab8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
